In [1]:
'''
Load and plot chipod, adcp, and thermistor data from moorings. Use modules in modview only.
- - - As of september 22 2021, this is using an outdated version of modview.loader
- - - - - - most recent development is only stored locally in kuxan-suum.
- - - Mooring IDs will be used as example to define the main properties and relations of 
- - - - - - modview-compatible databases of datasets. 
--- JA update: use sqlite3 and the database(LoaderDB3.db) to create obj_id and obj_paths
'''

import numpy as np; import xarray as xr; import pandas as pd;
import matplotlib.pyplot as plt; import matplotlib as mpl;
import sys; import scipy.io as sio

# Figure out how to import local package on Juno (iPad). 
# For PC or Linux:
modview_path = '/mnt/sda1/PhysOc/modview/modview';
sys.path.append(modview_path)
import loader
from DataBase_functions import *

# Begin by creating an instance of the assemble class. 

# Desired limits  = {'t0':'2018 Sep 15','t1':'2019 Oct 31'} are on table OBJ_ID;

#sio1_data_dir = '/media/mydrive/SciData/PISTON/Moorings/SIO1/'; 
sio1_data_dir = '/mnt/sda1/SciData/PISTON/Moorings/SIO1/';


####
# Old lines (Hard-code)
# sio1_adcp_paths = ['SIO1_4021.nc','SIO1_11181.nc'];
# sio1_paths = {'chipod':sio1_data_dir+'sio1_chipod.mat','chiflags':sio1_data_dir+'SIO1_chipod_finflag.csv',
#              'ADCP':[sio1_data_dir + 'ADCP/' + kk for kk in sio1_adcp_paths],
#              'Temperature':sio1_data_dir + 'SIO1_Tgrid.mat'};
# sio1_id = {'name':'SIO1','project':'PISTON','type':'Mooring','limits':sio1_limits};
####

sio1_id = id_maker(t0 = '2018 Sep 15')
sio1_id

[{'name': 'SIO1',
  'type': 'MOORING',
  'project': 'PISTON',
  'COMBINER_ID': 102,
  'limits': {'t0': '2018 Sep 15',
   't1': '2019 Oct 31',
   'lon': None,
   'lat': None}}]

In [3]:
Data = Data_extraction(sio1_id,0)
Data

[{'COMBINER_ID': 102,
  'DATA_OBJ': 201,
  'Source': 'chipod',
  'Filepath': 'sio1_chipod.mat'},
 {'COMBINER_ID': 102,
  'DATA_OBJ': 202,
  'Source': 'chiflag',
  'Filepath': 'SIO1_chipod_finflag.csv'},
 {'COMBINER_ID': 102,
  'DATA_OBJ': 301,
  'Source': 'ADCP',
  'Filepath': 'SIO1_4021.nc'},
 {'COMBINER_ID': 102,
  'DATA_OBJ': 302,
  'Source': 'ADCP',
  'Filepath': 'SIO1_11181.nc'},
 {'COMBINER_ID': 102,
  'DATA_OBJ': 401,
  'Source': 'Temperature',
  'Filepath': 'SIO1_Tgrid.mat'}]

In [4]:
sio1_paths = path_maker(Data,sio1_data_dir)
sio1_paths

{'chipod': ['/mnt/sda1/SciData/PISTON/Moorings/SIO1/sio1_chipod.mat'],
 'chiflag': ['/mnt/sda1/SciData/PISTON/Moorings/SIO1/SIO1_chipod_finflag.csv'],
 'ADCP': ['/mnt/sda1/SciData/PISTON/Moorings/SIO1/ADCP/SIO1_4021.nc',
  '/mnt/sda1/SciData/PISTON/Moorings/SIO1/ADCP/SIO1_11181.nc'],
 'Temperature': ['/mnt/sda1/SciData/PISTON/Moorings/SIO1/SIO1_Tgrid.mat']}

In [5]:
SIO1 = loader.assemble(sio1_id[0],sio1_paths) ## Tuve que agregar el 0 extra

# With the instance created, now use mat tools in loader to load chipod data
# replace with loadmat from updated version of loader
#SIO1.vars['matfile'] = loader.readmat(SIO1.files['chipod'], ['eps','N2','time','Jq'], structures='none');

chi_matfile = sio.loadmat(SIO1.files['chipod']);

# Get processing flags 
chi_flags = pd.read_csv(SIO1.paths['chiflags']); 
# Then it's all about extracting cells from the variable sio1_chipod in chi_matfile, turning those into pd.dfs


# After matfile has been loaded, access SIO1_chipod structure 
# there you can get 